In [2]:
import sagemaker
import boto3
sess = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket()
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='AmazonSageMaker-ExecutionRole-20231030T210397')['Role']['Arn']
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

Couldn't call 'get_role' to get Role ARN from role name ravi_tej to get Role path.


sagemaker role arn: arn:aws:iam::005418323977:role/service-role/AmazonSageMaker-ExecutionRole-20231030T210397
sagemaker bucket: sagemaker-ap-south-1-005418323977
sagemaker session region: ap-south-1


In [25]:
from transformers import AutoTokenizer
tokenizer_model_id = 'teknium/OpenHermes-2.5-Mistral-7B'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_id, trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="1.1.0",
  session=sess,
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.ap-south-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi1.1.0-gpu-py39-cu118-ubuntu20.04


In [4]:
import boto3
import os

# Initialize a boto3 S3 client
s3 = boto3.client('s3')

In [5]:
model_s3_path = 's3://sagemaker-ap-south-1-005418323977/huggingface-qlora-teknium-OpenHermes-2--2024-02-06-11-57-39-842/output/model/'

In [7]:
import json
from sagemaker.huggingface import HuggingFaceModel
# sagemaker config
instance_type = "ml.g5.xlarge"
number_of_gpu = 1
health_check_timeout = 300

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # path to where sagemaker stores the model
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(3584), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(4096), # Max length of the generation (including input text)
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  role=role,
  image_uri=llm_image,
  model_data={'S3DataSource':{'S3Uri': model_s3_path,'S3DataType': 'S3Prefix','CompressionType': 'None'}},
  env=config
)

In [8]:
finetune_id= 'MangoPurpleValley'

In [9]:
# Deploy model to an endpoint
# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy
llm = llm_model.deploy(
    endpoint_name = f"OpenHermes-Search-{finetune_id.replace('_','-')}",
  initial_instance_count=1,
  instance_type=instance_type,
  container_startup_health_check_timeout=health_check_timeout, # 10 minutes to be able to load the model
)

----------!

In [10]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [11]:
from src.HybridSearchService import HybridSearchService
from src.QueryService import QueryService
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation
from datetime import datetime

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [28]:
smr = sess.boto_session.client("sagemaker-runtime")

In [336]:
query = "Adani Gas's natural gas distribution plans for 2024"

In [433]:
QueryService.understand_query_using_llm('sensex today')

query understanding llm in 1.0022270679473877 seconds


(True, 'high', 'Sensex value on 08 February 2024')

In [337]:
results_object = HybridSearchService.perform_hybrid_reranked_search(query_text =query, recency_importance='medium')
top_hits = [x['article_id'] for x in results_object['searchArticleIds']][:3]

took 0.151 seconds to try and retrieve embeddings for search query Adani Gas's natural gas distribution plans for 2024
took 1.533 seconds to retrieve nearest 250 sematic items for search query Adani Gas's natural gas distribution plans for 2024
took 2.391 seconds to retrieve nearest 250 exact items for search query Adani Gas's natural gas distribution plans for 2024
took 3.301 seconds to fetch the metadata from mongo for search query Adani Gas's natural gas distribution plans for 2024
took 3.342 seconds to calculate recency weighted items Adani Gas's natural gas distribution plans for 2024
took 5.11 seconds to rerank the results for search query Adani Gas's natural gas distribution plans for 2024


In [338]:
top_hits

['65c1a0d9c119ff6e57a3f2a7',
 '65c04f53c119ff6e57a3eeec',
 '65bb0960f3aa2a77a58bf9bc']

In [339]:
unformatted_full_prompt = SummaryService.create_prompt_for_query_references(search_query=query,references_article_ids=top_hits, today_date=None)

In [403]:
parameters = {
    "do_sample": True,
    "top_p": 0.99,
    "temperature": 0.2,
    "max_new_tokens": 512,
    "repetition_penalty": 1.1,
    "stop": ["###", "</s>", tokenizer.eos_token],
    'seed': 86
}

In [404]:
def split_prompt(input_prompt):
    x, y = input_prompt.split('|| user_query')
    y = '|| user_query' + y
    return x, y

In [405]:
sys_prompt, user_content = split_prompt(unformatted_full_prompt)

In [426]:
from datetime import datetime
import pytz
from zoneinfo import ZoneInfo

In [427]:
k = datetime.now(tz=ZoneInfo('Asia/Kolkata')).strftime('%Y-%m-%dT%H:%M:%S%z')

In [428]:
k

'2024-02-08T08:30:48+0530'

In [429]:
datetime_obj = datetime.strptime(k, '%Y-%m-%dT%H:%M:%S%z')

In [431]:
datetime_obj.strftime('%d %B %Y')

'08 February 2024'

In [406]:
messages = [{"role": "system", "content": sys_prompt},
            {"role": "user", "content": user_content}]
context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))

In [407]:
request = {"inputs": context_prompt, "parameters": parameters, "stream": True}

In [408]:
import time

In [434]:
import tiktoken

In [441]:
import requests

2024-02-08 10:54:50,317| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [ ]:
re

In [410]:
context_prompt

"<|im_start|> system\nYou're the lead editor for a financial and business website. You answer user queries by using relevant articles that you've found by strictly following the below rules:\n*valid_queries: questions about finance, business, or relevant entities like India, RBI, Reliance, Fed etc\n*invalid_queries: non financial or business questions or inappropriate content\nresponse_instructions:\n1.query is valid and answerable with the articles you found - summarize the information clearly and concisely with a headline and followed by 3 to 6 points with following guidelines: \nPrefix the point with a relevant label one emoji for that point. The points should overall capture the essence of a good response for the user query, but at the same time as brief as possible - a few words to a maximum of 1 sentence per point. Start with the most important point for the query. The labels should be human readable avoid use of _ or programming symbols. Headline should be between 5 to 15 words 

In [409]:
t = time.time()
resp = smr.invoke_endpoint_with_response_stream(
    EndpointName=f"OpenHermes-Search-MangoPurpleValley",
    Body=json.dumps(request),
    ContentType="application/json",
)
# print(f'done in {time.time() - t} seconds')
# k = resp['Body'].read()
for token in TokenIterator(resp["Body"]):
    print(token, end="")

**Adani Total Gas's Natural Gas Distribution Plans for 2024**<br><br>📈 **Profit Growth**: 18% increase in Q3FY24 net profit to ₹176.64 cr.<br><br>💡 **Market Expansion**: 21% increase in gas sales volume to 224 MMSCM.<br><br>🛢️ **EBITDA Boost**: EBITDA up 26% to ₹301 cr., driven by volume and price strategies.<br><br>⚙️ **Project Progress**: First phase of 600 TPD biomass plant to be commissioned by March 2024.<br><br>🚚 **LNG for Transports**: First LNG retail outlet to be operational by July 2024.<br><br>🔍 **Financial Health**: EBITDA margin remains healthy, with a 20.2% margin in Q3FY24. <br><br>Please note, while the above points give a glimpse of the company's current and future plans, it's important to conduct further research and consult an expert before making any investment decisions.<|im_end|>

In [217]:
a = '''�'''

In [166]:
tokenizer.encode(a)

[1, 28705, 29137]

In [191]:
tokenizer.decode([29137])

'�'

In [198]:
valid = '💡💼🏦📈🔍📊'

In [202]:
tokenizer.decode([164])

'�'

In [195]:
tokenizer.encode(valid)

[1,
 28705,
 243,
 162,
 149,
 164,
 243,
 162,
 149,
 191,
 243,
 162,
 146,
 169,
 243,
 162,
 150,
 139,
 243,
 162,
 151,
 144,
 243,
 162,
 150,
 141]

In [163]:
b = 'article'

In [164]:
tokenizer.encode(b)

[1, 5447]

In [443]:
import numpy as np

In [446]:
k = [(1,'a'),(-1,'b')]

In [447]:
k.sort()

2024-02-08 14:42:05,092| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active


In [454]:
if np.array([1,3,5])< 10:
    print('a')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

2024-02-08 14:45:06,425| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:45:16,493| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:45:26,556| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:45:36,625| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ss

In [451]:
np.diff([2,3],[4,5])

TypeError: '<' not supported between instances of 'list' and 'int'

2024-02-08 14:43:55,922| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:44:05,991| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:44:16,072| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:44:26,147| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ss

In [448]:
k

[(-1, 'b'), (1, 'a')]

2024-02-08 14:42:15,175| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:42:25,242| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:42:35,322| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:42:45,396| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ss

In [444]:
np.sort([(1,'a'),(-1,'b')])

array([['1', 'a'],
       ['-1', 'b']], dtype='<U21')

2024-02-08 14:39:23,874| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:39:33,946| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:39:44,014| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ssh error: SSH session not active
2024-02-08 14:39:54,089| ERROR   | Could not establish connection from local ('127.0.0.1', 3793) to remote ('insight-beta-article-db2.chmaiagjipqy.ap-south-1.docdb.amazonaws.com', 27017) side of the tunnel: open new channel ss

In [160]:
json.loads(k)[0]['generated_text']

'**Top 3 Dividend ETFs to Invest in 2024**<br><br>💼 **Best ETF to buy now**: VBG is the top-rated ETF, offering a 1.20% annual yield and a 0.05% annual expense ratio. <br><br>💼 **Top dividend ETF in the market**: VG is the top-rated ETF in the category, with a 1.15% annual yield and a 0.05% annual expense ratio. <br><br>�� helping to manage the market: The S&P 500 is at a record high, but the market is still a good time to invest, as the market is on a bull market and the market is going up in the long run.  <br><br>**Note**: The above information is based on the most recent data available as of the articles date, and investment advice should be sought from a financial expert before making investment decisions.'

In [31]:
import io
import json

class TokenIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            if line and line[-1] == ord("\n"):
                self.read_pos += len(line) + 1
                full_line = line[:-1].decode("utf-8")
                line_data = json.loads(full_line.lstrip("data:").rstrip("/n"))
                return line_data["token"]["text"]
            chunk = next(self.byte_iterator)
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk["PayloadPart"]["Bytes"])